## Train Disruptions in the Netherlands

<img alt="ns-delay.jpg" src="ns-delay.jpg" width="1000"/>

### Exploring data from Rijden de Treinen using [Kotlin for Data Analysis](https://kotlinlang.org/docs/data-analysis-overview.html)

Data from https://www.rijdendetreinen.nl/en/open-data/

Let's find out together:
- What causes delays?
- What's the longest delay in 2023 and where did it occur?
- Which track or station had the most delays in 2023?
- Do I get to complain about Dutch trains in live demos? (I came by car)

[disruptions 2023 CSV](data/disruptions/disruptions-2023.csv)

In [65]:
%use dataframe

In [66]:
// reading the csv by dragAndDrop
val disruptions2023 = DataFrame.readCSV("data/disruptions/disruptions-2023.csv", delimiter = ',')
disruptions2023

rdt_id,ns_lines,rdt_lines,rdt_lines_id,rdt_station_names,rdt_station_codes,cause_nl,cause_en,statistical_cause_nl,statistical_cause_en,cause_group,start_time,end_time,duration_minutes
45999,Amsterdam-Rotterdam-Brussel (HSL),Amsterdam Centraal - Schiphol Airport...,2432.000000,"Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",wisselstoring,points failure,wisselstoring,points failure,infrastructure,2023-01-01T08:19:26,2023-01-01T22:43:08,864
46000,Zwolle-Leeuwarden,Leeuwarden - Zwolle,160.000000,"Heerenveen,Wolvega,Heerenveen IJsstadion","HR, WV, HRY",dier op het spoor,an animal on the railway track,dier op het spoor,an animal on the railway track,external,2023-01-01T10:31:49,2023-01-01T10:56:17,24
46001,Heerlen-Aachen Hbf,Aachen Hbf - Heerlen,130.000000,"Aachen Hbf,Eygelshoven Markt,Heerlen,...","AHBF, EGHM, HRL, HRLK, HZ, LG, AW",beperkingen in de materieelinzet,problems with the rolling stock,beperkingen in de materieelinzet,problems with the rolling stock,rolling stock,2023-01-01T13:19:24,2023-01-02T00:02:39,643
46002,Zutphen-Winterswijk,Winterswijk - Zutphen,83.000000,"Vorden,Zutphen","VD, ZP",aanrijding,collision,aanrijding,collision,accidents,2023-01-01T17:15:22,2023-01-01T20:14:23,179
46003,Heerlen-Aachen Hbf,Aachen Hbf - Heerlen,130.000000,"Aachen Hbf,Eygelshoven Markt,Heerlen,...","AHBF, EGHM, HRL, HRLK, HZ, LG, AW",beperkingen in de materieelinzet,problems with the rolling stock,beperkingen in de materieelinzet,problems with the rolling stock,rolling stock,2023-01-02T05:57:27,2023-01-03T02:07:13,1210
46004,Amersfoort-Ede-Wageningen,Amersfoort - Ede-Wageningen,47.000000,"Amersfoort Centraal,Barneveld Centrum...","AMF, BNC, BNN, ED, EDC, LTN, HVL, BNZ",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T06:36:39,2023-01-02T07:28:16,52
46005,Dordrecht-Breda; Dordrecht-Roosendaal,"Breda - Dordrecht, Dordrecht - Roosen...",170171.000000,"Dordrecht,Dordrecht Zuid,Lage Zwaluwe","DDR, DDZD, ZLW",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T07:31:33,2023-01-02T08:09:37,38
46006,'s-Hertogenbosch-Tilburg,'s-Hertogenbosch - Tilburg,69.000000,"'s-Hertogenbosch,Tilburg","HT, TB",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T11:33:15,2023-01-02T11:44:27,11
46007,Rotterdam-Breda (HSL),Breda - Rotterdam Centraal (HSL),15.000000,"Breda,Rotterdam Centraal","BD, RTD",gestrande trein,stranded train,gestrande trein,stranded train,rolling stock,2023-01-02T11:50:11,2023-01-02T12:25:39,35
46008,Amsterdam-Schiphol-Rotterdam (HSL),Amsterdam Centraal - Schiphol Airport...,2432.000000,"Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T12:40:11,2023-01-02T13:08:08,28


In [67]:
disruptions2023.schema()

rdt_id: Int
ns_lines: String
rdt_lines: String?
rdt_lines_id: Double?
rdt_station_names: String?
rdt_station_codes: String?
cause_nl: String
cause_en: String
statistical_cause_nl: String
statistical_cause_en: String
cause_group: String
start_time: kotlinx.datetime.LocalDateTime
end_time: kotlinx.datetime.LocalDateTime?
duration_minutes: Int?

 Looking at the schema, we can see it mostly parsed the data correctly.
 `rdt_lines_id: Double?` is a mistake though.
 
From the website: "These are the IDs of the lines linked to a disruption by Rijden de Treinen, separated by a comma."
Understandably, `"24,32"` is parsed as a `Double` instead of `String`. Let's try to nudge it into the right direction when reading the data
by supplying it with a manual type for this column.

Let's also rename it to camel case while we're at it.

In [68]:
val disruptions2023 = DataFrame.readCSV(
    fileOrUrl = "data/disruptions/disruptions-2023.csv",
    delimiter = ',',
    colTypes = mapOf("rdt_lines_id" to ColType.String),
).renameToCamelCase()

disruptions2023

rdtId,nsLines,rdtLines,rdtLinesId,rdtStationNames,rdtStationCodes,causeNl,causeEn,statisticalCauseNl,statisticalCauseEn,causeGroup,startTime,endTime,durationMinutes
45999,Amsterdam-Rotterdam-Brussel (HSL),Amsterdam Centraal - Schiphol Airport...,"24,32","Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",wisselstoring,points failure,wisselstoring,points failure,infrastructure,2023-01-01T08:19:26,2023-01-01T22:43:08,864
46000,Zwolle-Leeuwarden,Leeuwarden - Zwolle,160,"Heerenveen,Wolvega,Heerenveen IJsstadion","HR, WV, HRY",dier op het spoor,an animal on the railway track,dier op het spoor,an animal on the railway track,external,2023-01-01T10:31:49,2023-01-01T10:56:17,24
46001,Heerlen-Aachen Hbf,Aachen Hbf - Heerlen,130,"Aachen Hbf,Eygelshoven Markt,Heerlen,...","AHBF, EGHM, HRL, HRLK, HZ, LG, AW",beperkingen in de materieelinzet,problems with the rolling stock,beperkingen in de materieelinzet,problems with the rolling stock,rolling stock,2023-01-01T13:19:24,2023-01-02T00:02:39,643
46002,Zutphen-Winterswijk,Winterswijk - Zutphen,83,"Vorden,Zutphen","VD, ZP",aanrijding,collision,aanrijding,collision,accidents,2023-01-01T17:15:22,2023-01-01T20:14:23,179
46003,Heerlen-Aachen Hbf,Aachen Hbf - Heerlen,130,"Aachen Hbf,Eygelshoven Markt,Heerlen,...","AHBF, EGHM, HRL, HRLK, HZ, LG, AW",beperkingen in de materieelinzet,problems with the rolling stock,beperkingen in de materieelinzet,problems with the rolling stock,rolling stock,2023-01-02T05:57:27,2023-01-03T02:07:13,1210
46004,Amersfoort-Ede-Wageningen,Amersfoort - Ede-Wageningen,47,"Amersfoort Centraal,Barneveld Centrum...","AMF, BNC, BNN, ED, EDC, LTN, HVL, BNZ",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T06:36:39,2023-01-02T07:28:16,52
46005,Dordrecht-Breda; Dordrecht-Roosendaal,"Breda - Dordrecht, Dordrecht - Roosen...","170,171","Dordrecht,Dordrecht Zuid,Lage Zwaluwe","DDR, DDZD, ZLW",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T07:31:33,2023-01-02T08:09:37,38
46006,'s-Hertogenbosch-Tilburg,'s-Hertogenbosch - Tilburg,69,"'s-Hertogenbosch,Tilburg","HT, TB",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T11:33:15,2023-01-02T11:44:27,11
46007,Rotterdam-Breda (HSL),Breda - Rotterdam Centraal (HSL),15,"Breda,Rotterdam Centraal","BD, RTD",gestrande trein,stranded train,gestrande trein,stranded train,rolling stock,2023-01-02T11:50:11,2023-01-02T12:25:39,35
46008,Amsterdam-Schiphol-Rotterdam (HSL),Amsterdam Centraal - Schiphol Airport...,"24,32","Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T12:40:11,2023-01-02T13:08:08,28


In [69]:
disruptions2023.schema()

rdtId: Int
nsLines: String
rdtLines: String?
rdtLinesId: String?
rdtStationNames: String?
rdtStationCodes: String?
causeNl: String
causeEn: String
statisticalCauseNl: String
statisticalCauseEn: String
causeGroup: String
startTime: kotlinx.datetime.LocalDateTime
endTime: kotlinx.datetime.LocalDateTime?
durationMinutes: Int?

Now the schema looks better! One of the best things about using DataFrame in notebooks
is that in between cell calls type-safe accessors are generated for you!

In [70]:
disruptions2023.rdtLinesId

rdtLinesId
"24,32"
160
130
83
130
47
"170,171"
69
15
"24,32"


We can actually make this hidden process visible by tracking all code that's executed under the hood.

Libraries for the Kotlin Jupyter kernel and notebooks can be very powerful as you can see!

In [71]:
%trackExecution
val dataFrame = dataFrameOf("a", "b")(1, 2)

Executing:

val dataFrame = dataFrameOf("a", "b")(1, 2)

Executing:
(dataFrame as org.jetbrains.kotlinx.dataframe.DataFrame<*>).cast<Line_15_jupyter._DataFrameType2>()
Executing:
val dataFrame = res140


In [72]:
%trackExecution off

In [73]:
val a = dataFrame.a
val b = dataFrame.b

a

a
1


Anyway, let's get back to our data!

Let's make our data easier to work with:
- We already renamed to camelCase
- Remove Dutch columns in favor of English ones
- Remove NS and `cause` columns (in favor of rdt columns and statisticalCause respectively)
- Drop rows where durationMinutes is `null`
- Add helper columns for just the `date: LocalDate` and `duration: kotlin.time.Duration` for easier viewing and plotting
- Parse comma-split columns as lists

For an overview, check out [DataFrame Operations](https://kotlin.github.io/dataframe/operations.html)

In [74]:
// before
disruptions2023

rdtId,nsLines,rdtLines,rdtLinesId,rdtStationNames,rdtStationCodes,causeNl,causeEn,statisticalCauseNl,statisticalCauseEn,causeGroup,startTime,endTime,durationMinutes
45999,Amsterdam-Rotterdam-Brussel (HSL),Amsterdam Centraal - Schiphol Airport...,"24,32","Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",wisselstoring,points failure,wisselstoring,points failure,infrastructure,2023-01-01T08:19:26,2023-01-01T22:43:08,864
46000,Zwolle-Leeuwarden,Leeuwarden - Zwolle,160,"Heerenveen,Wolvega,Heerenveen IJsstadion","HR, WV, HRY",dier op het spoor,an animal on the railway track,dier op het spoor,an animal on the railway track,external,2023-01-01T10:31:49,2023-01-01T10:56:17,24
46001,Heerlen-Aachen Hbf,Aachen Hbf - Heerlen,130,"Aachen Hbf,Eygelshoven Markt,Heerlen,...","AHBF, EGHM, HRL, HRLK, HZ, LG, AW",beperkingen in de materieelinzet,problems with the rolling stock,beperkingen in de materieelinzet,problems with the rolling stock,rolling stock,2023-01-01T13:19:24,2023-01-02T00:02:39,643
46002,Zutphen-Winterswijk,Winterswijk - Zutphen,83,"Vorden,Zutphen","VD, ZP",aanrijding,collision,aanrijding,collision,accidents,2023-01-01T17:15:22,2023-01-01T20:14:23,179
46003,Heerlen-Aachen Hbf,Aachen Hbf - Heerlen,130,"Aachen Hbf,Eygelshoven Markt,Heerlen,...","AHBF, EGHM, HRL, HRLK, HZ, LG, AW",beperkingen in de materieelinzet,problems with the rolling stock,beperkingen in de materieelinzet,problems with the rolling stock,rolling stock,2023-01-02T05:57:27,2023-01-03T02:07:13,1210
46004,Amersfoort-Ede-Wageningen,Amersfoort - Ede-Wageningen,47,"Amersfoort Centraal,Barneveld Centrum...","AMF, BNC, BNN, ED, EDC, LTN, HVL, BNZ",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T06:36:39,2023-01-02T07:28:16,52
46005,Dordrecht-Breda; Dordrecht-Roosendaal,"Breda - Dordrecht, Dordrecht - Roosen...","170,171","Dordrecht,Dordrecht Zuid,Lage Zwaluwe","DDR, DDZD, ZLW",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T07:31:33,2023-01-02T08:09:37,38
46006,'s-Hertogenbosch-Tilburg,'s-Hertogenbosch - Tilburg,69,"'s-Hertogenbosch,Tilburg","HT, TB",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T11:33:15,2023-01-02T11:44:27,11
46007,Rotterdam-Breda (HSL),Breda - Rotterdam Centraal (HSL),15,"Breda,Rotterdam Centraal","BD, RTD",gestrande trein,stranded train,gestrande trein,stranded train,rolling stock,2023-01-02T11:50:11,2023-01-02T12:25:39,35
46008,Amsterdam-Schiphol-Rotterdam (HSL),Amsterdam Centraal - Schiphol Airport...,"24,32","Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",defecte trein,broken down train,defecte trein,broken down train,rolling stock,2023-01-02T12:40:11,2023-01-02T13:08:08,28


In [75]:
import kotlin.time.Duration.Companion.minutes

val df1 = disruptions2023

    // we remove nsLines, dutch columns, and causeEn (as statisticalCauseEn is better according to the docs)
    .remove { nsLines and nameEndsWith("Nl") and causeEn }

    // let's also remove some rows where durationMinutes == null
    .dropNulls { durationMinutes }
    
    // Parsing minutes into kotlin.time.Duration and creating an extra date column
    .add {
        "duration" from { durationMinutes!!.minutes }
        "date" from { startTime.date }
    }

    // renaming columns to remove "rdt" and "En" from the beginning and end
    .rename { all() }.into {
        it.name
            .removePrefix("rdt").replaceFirstChar { it.lowercase() }
            .removeSuffix("En")
    }

df1

id,lines,linesId,stationNames,stationCodes,statisticalCause,causeGroup,startTime,endTime,durationMinutes,duration,date
45999,Amsterdam Centraal - Schiphol Airport...,"24,32","Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",points failure,infrastructure,2023-01-01T08:19:26,2023-01-01T22:43:08,864,14h 24m,2023-01-01
46000,Leeuwarden - Zwolle,160,"Heerenveen,Wolvega,Heerenveen IJsstadion","HR, WV, HRY",an animal on the railway track,external,2023-01-01T10:31:49,2023-01-01T10:56:17,24,24m,2023-01-01
46001,Aachen Hbf - Heerlen,130,"Aachen Hbf,Eygelshoven Markt,Heerlen,...","AHBF, EGHM, HRL, HRLK, HZ, LG, AW",problems with the rolling stock,rolling stock,2023-01-01T13:19:24,2023-01-02T00:02:39,643,10h 43m,2023-01-01
46002,Winterswijk - Zutphen,83,"Vorden,Zutphen","VD, ZP",collision,accidents,2023-01-01T17:15:22,2023-01-01T20:14:23,179,2h 59m,2023-01-01
46003,Aachen Hbf - Heerlen,130,"Aachen Hbf,Eygelshoven Markt,Heerlen,...","AHBF, EGHM, HRL, HRLK, HZ, LG, AW",problems with the rolling stock,rolling stock,2023-01-02T05:57:27,2023-01-03T02:07:13,1210,20h 10m,2023-01-02
46004,Amersfoort - Ede-Wageningen,47,"Amersfoort Centraal,Barneveld Centrum...","AMF, BNC, BNN, ED, EDC, LTN, HVL, BNZ",broken down train,rolling stock,2023-01-02T06:36:39,2023-01-02T07:28:16,52,52m,2023-01-02
46005,"Breda - Dordrecht, Dordrecht - Roosen...","170,171","Dordrecht,Dordrecht Zuid,Lage Zwaluwe","DDR, DDZD, ZLW",broken down train,rolling stock,2023-01-02T07:31:33,2023-01-02T08:09:37,38,38m,2023-01-02
46006,'s-Hertogenbosch - Tilburg,69,"'s-Hertogenbosch,Tilburg","HT, TB",broken down train,rolling stock,2023-01-02T11:33:15,2023-01-02T11:44:27,11,11m,2023-01-02
46007,Breda - Rotterdam Centraal (HSL),15,"Breda,Rotterdam Centraal","BD, RTD",stranded train,rolling stock,2023-01-02T11:50:11,2023-01-02T12:25:39,35,35m,2023-01-02
46008,Amsterdam Centraal - Schiphol Airport...,"24,32","Amsterdam Centraal,Amsterdam Lelylaan...","ASD, ASDL, ASS, RTD, SHL",broken down train,rolling stock,2023-01-02T12:40:11,2023-01-02T13:08:08,28,28m,2023-01-02


Almost perfect! However, we still have some list-like columns. We can split those into lists to make them more manageable.

In [76]:
val df2 = df1
    // splitting lines, linesId, stationNames, stationCodes by ","
    .split {
        cols(lines, linesId, stationNames, stationCodes)
    }.by(",").inplace()
    
    // converting linesId from List<String> to List<Int>
    .convert { linesId.cast<List<String>>() }.with { it.map { it.toInt() } }

df2

id,lines,linesId,stationNames,stationCodes,statisticalCause,causeGroup,startTime,endTime,durationMinutes,duration,date
45999,[Amsterdam Centraal - Schiphol Airpor...,"[24, 32]","[Amsterdam Centraal, Amsterdam Lelyla...","[ASD, ASDL, ASS, RTD, SHL]",points failure,infrastructure,2023-01-01T08:19:26,2023-01-01T22:43:08,864,14h 24m,2023-01-01
46000,[Leeuwarden - Zwolle],[160],"[Heerenveen, Wolvega, Heerenveen IJss...","[HR, WV, HRY]",an animal on the railway track,external,2023-01-01T10:31:49,2023-01-01T10:56:17,24,24m,2023-01-01
46001,[Aachen Hbf - Heerlen],[130],"[Aachen Hbf, Eygelshoven Markt, Heerl...","[AHBF, EGHM, HRL, HRLK, HZ, LG, AW]",problems with the rolling stock,rolling stock,2023-01-01T13:19:24,2023-01-02T00:02:39,643,10h 43m,2023-01-01
46002,[Winterswijk - Zutphen],[83],"[Vorden, Zutphen]","[VD, ZP]",collision,accidents,2023-01-01T17:15:22,2023-01-01T20:14:23,179,2h 59m,2023-01-01
46003,[Aachen Hbf - Heerlen],[130],"[Aachen Hbf, Eygelshoven Markt, Heerl...","[AHBF, EGHM, HRL, HRLK, HZ, LG, AW]",problems with the rolling stock,rolling stock,2023-01-02T05:57:27,2023-01-03T02:07:13,1210,20h 10m,2023-01-02
46004,[Amersfoort - Ede-Wageningen],[47],"[Amersfoort Centraal, Barneveld Centr...","[AMF, BNC, BNN, ED, EDC, LTN, HVL, BNZ]",broken down train,rolling stock,2023-01-02T06:36:39,2023-01-02T07:28:16,52,52m,2023-01-02
46005,"[Breda - Dordrecht, Dordrecht - Roose...","[170, 171]","[Dordrecht, Dordrecht Zuid, Lage Zwal...","[DDR, DDZD, ZLW]",broken down train,rolling stock,2023-01-02T07:31:33,2023-01-02T08:09:37,38,38m,2023-01-02
46006,['s-Hertogenbosch - Tilburg],[69],"['s-Hertogenbosch, Tilburg]","[HT, TB]",broken down train,rolling stock,2023-01-02T11:33:15,2023-01-02T11:44:27,11,11m,2023-01-02
46007,[Breda - Rotterdam Centraal (HSL)],[15],"[Breda, Rotterdam Centraal]","[BD, RTD]",stranded train,rolling stock,2023-01-02T11:50:11,2023-01-02T12:25:39,35,35m,2023-01-02
46008,[Amsterdam Centraal - Schiphol Airpor...,"[24, 32]","[Amsterdam Centraal, Amsterdam Lelyla...","[ASD, ASDL, ASS, RTD, SHL]",broken down train,rolling stock,2023-01-02T12:40:11,2023-01-02T13:08:08,28,28m,2023-01-02


In [77]:
df2.schema()

id: Int
lines: List<String>
linesId: List<Int>
stationNames: List<String>
stationCodes: List<String>
statisticalCause: String
causeGroup: String
startTime: kotlinx.datetime.LocalDateTime
endTime: kotlinx.datetime.LocalDateTime
durationMinutes: Int
duration: time.Duration
date: kotlinx.datetime.LocalDate

Done! Now let's get to work!

Remember, we wanted to find:

- What's the longest delay in 2023 and where did it occur?
- What causes delays?
- Which track or station had the most delays in 2023?
- Do I get to complain about Dutch trains in live demos? (I came by car)


## Longest delay in 2023?

In [78]:
df2

id,lines,linesId,stationNames,stationCodes,statisticalCause,causeGroup,startTime,endTime,durationMinutes,duration,date
45999,[Amsterdam Centraal - Schiphol Airpor...,"[24, 32]","[Amsterdam Centraal, Amsterdam Lelyla...","[ASD, ASDL, ASS, RTD, SHL]",points failure,infrastructure,2023-01-01T08:19:26,2023-01-01T22:43:08,864,14h 24m,2023-01-01
46000,[Leeuwarden - Zwolle],[160],"[Heerenveen, Wolvega, Heerenveen IJss...","[HR, WV, HRY]",an animal on the railway track,external,2023-01-01T10:31:49,2023-01-01T10:56:17,24,24m,2023-01-01
46001,[Aachen Hbf - Heerlen],[130],"[Aachen Hbf, Eygelshoven Markt, Heerl...","[AHBF, EGHM, HRL, HRLK, HZ, LG, AW]",problems with the rolling stock,rolling stock,2023-01-01T13:19:24,2023-01-02T00:02:39,643,10h 43m,2023-01-01
46002,[Winterswijk - Zutphen],[83],"[Vorden, Zutphen]","[VD, ZP]",collision,accidents,2023-01-01T17:15:22,2023-01-01T20:14:23,179,2h 59m,2023-01-01
46003,[Aachen Hbf - Heerlen],[130],"[Aachen Hbf, Eygelshoven Markt, Heerl...","[AHBF, EGHM, HRL, HRLK, HZ, LG, AW]",problems with the rolling stock,rolling stock,2023-01-02T05:57:27,2023-01-03T02:07:13,1210,20h 10m,2023-01-02
46004,[Amersfoort - Ede-Wageningen],[47],"[Amersfoort Centraal, Barneveld Centr...","[AMF, BNC, BNN, ED, EDC, LTN, HVL, BNZ]",broken down train,rolling stock,2023-01-02T06:36:39,2023-01-02T07:28:16,52,52m,2023-01-02
46005,"[Breda - Dordrecht, Dordrecht - Roose...","[170, 171]","[Dordrecht, Dordrecht Zuid, Lage Zwal...","[DDR, DDZD, ZLW]",broken down train,rolling stock,2023-01-02T07:31:33,2023-01-02T08:09:37,38,38m,2023-01-02
46006,['s-Hertogenbosch - Tilburg],[69],"['s-Hertogenbosch, Tilburg]","[HT, TB]",broken down train,rolling stock,2023-01-02T11:33:15,2023-01-02T11:44:27,11,11m,2023-01-02
46007,[Breda - Rotterdam Centraal (HSL)],[15],"[Breda, Rotterdam Centraal]","[BD, RTD]",stranded train,rolling stock,2023-01-02T11:50:11,2023-01-02T12:25:39,35,35m,2023-01-02
46008,[Amsterdam Centraal - Schiphol Airpor...,"[24, 32]","[Amsterdam Centraal, Amsterdam Lelyla...","[ASD, ASDL, ASS, RTD, SHL]",broken down train,rolling stock,2023-01-02T12:40:11,2023-01-02T13:08:08,28,28m,2023-01-02



## What causes delays?

I'm actually quite interested in these causes and what makes up a "cause group".
Let's find all groups and see what causes are inside :)

Note the nested DataFrames :)

In [79]:
df2
    // group by causeGroup and get `valueCounts()` of statisticalCause
    .groupBy { causeGroup }.aggregate {
        statisticalCause.valueCounts() into "causes"
    }
    // sort descending by the number of rows in causes 
    .sortByDesc { 
        expr { getFrameColumn("causes").count() } 
    }

<iframe onload="o_resize_iframe_out_72()" style="width:100%;" class="result_container" id="iframe_out_72" frameBorder="0" srcdoc=" <html theme='dark'>
 <head>
 <style type="text/css">
 :root {
 --background: #fff;
 --background-odd: #f5f5f5;
 --background-hover: #d9edfd;
 --header-text-color: #474747;
 --text-color: #848484;
 --text-color-dark: #000;
 --text-color-medium: #737373;
 --text-color-pale: #b3b3b3;
 --inner-border-color: #aaa;
 --bold-border-color: #000;
 --link-color: #296eaa;
 --link-color-pale: #296eaa;
 --link-hover: #1a466c;
}

:root[theme="dark"], :root [data-jp-theme-light="false"], .dataframe_dark{
 --background: #303030;
 --background-odd: #3c3c3c;
 --background-hover: #464646;
 --header-text-color: #dddddd;
 --text-color: #b3b3b3;
 --text-color-dark: #dddddd;
 --text-color-medium: #b2b2b2;
 --text-color-pale: #737373;
 --inner-border-color: #707070;
 --bold-border-color: #777777;
 --link-color: #008dc0;
 --link-color-pale: #97e1fb;
 --link-hover: #00688e;
}

p.dataframe_description {
 color: var(--text-color-dark);
}

table.dataframe {
 font-family: "Helvetica Neue", Helvetica, Arial, sans-serif;
 font-size: 12px;
 background-color: var(--background);
 color: var(--text-color-dark);
 border: none;
 border-collapse: collapse;
}

table.dataframe th, td {
 padding: 6px;
 border: 1px solid transparent;
 text-align: left;
}

table.dataframe th {
 background-color: var(--background);
 color: var(--header-text-color);
}

table.dataframe td {
 vertical-align: top;
}

table.dataframe th.bottomBorder {
 border-bottom-color: var(--bold-border-color);
}

table.dataframe tbody > tr:nth-child(odd) {
 background: var(--background-odd);
}

table.dataframe tbody > tr:nth-child(even) {
 background: var(--background);
}

table.dataframe tbody > tr:hover {
 background: var(--background-hover);
}

table.dataframe a {
 cursor: pointer;
 color: var(--link-color);
 text-decoration: none;
}

table.dataframe tr:hover > td a {
 color: var(--link-color-pale);
}

table.dataframe a:hover {
 color: var(--link-hover);
 text-decoration: underline;
}

table.dataframe img {
 max-width: fit-content;
}

table.dataframe th.complex {
 background-color: var(--background);
 border: 1px solid var(--background);
}

table.dataframe .leftBorder {
 border-left-color: var(--inner-border-color);
}

table.dataframe .rightBorder {
 border-right-color: var(--inner-border-color);
}

table.dataframe .rightAlign {
 text-align: right;
}

table.dataframe .expanderSvg {
 width: 8px;
 height: 8px;
 margin-right: 3px;
}

table.dataframe .expander {
 display: flex;
 align-items: center;
}

/* formatting */

table.dataframe .null {
 color: var(--text-color-pale);
}

table.dataframe .structural {
 color: var(--text-color-medium);
 font-weight: bold;
}

table.dataframe .dataFrameCaption {
 font-weight: bold;
}

table.dataframe .numbers {
 color: var(--text-color-dark);
}

table.dataframe td:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}

table.dataframe tr:hover .formatted .structural, .null {
 color: var(--text-color-dark);
}


:root {
 --scroll-bg: #f5f5f5;
 --scroll-fg: #b3b3b3;
}
:root[theme="dark"], :root [data-jp-theme-light="false"]{
 --scroll-bg: #3c3c3c;
 --scroll-fg: #97e1fb;
}
body {
 scrollbar-color: var(--scroll-fg) var(--scroll-bg);
}
body::-webkit-scrollbar {
 width: 10px; /* Mostly for vertical scrollbars */
 height: 10px; /* Mostly for horizontal scrollbars */
}
body::-webkit-scrollbar-thumb {
 background-color: var(--scroll-fg);
}
body::-webkit-scrollbar-track {
 background-color: var(--scroll-bg);
}
 </style>
 </head>
 <body>
 <table class="dataframe" id="df_654311830"></table>

<p class="dataframe_description">DataFrame: rowsCount = 9, columnsCount = 2</p>

 </body>
 <script>
 (function () {
 window.DataFrame = window.DataFrame || new (function () {
 this.addTable = function (df) {
 let cols = df.cols;
 for (let i = 0; i < cols.length; i++) {
 for (let c of cols[i].children) {
 cols[c].parent = i;
 }
 }
 df.nrow = 0
 for (l

## Which line had the most delays?

To find the line with the most delays, we first need to explode the `lines` column
to get a separate row for each line, then group by the `lines` column and count how many rows 
we get per individual line. Finally, sort descending by count.

In [80]:
val byLines = df2
    .explode { lines }
    .groupBy { lines }

byLines.count()

lines,count
Amsterdam Centraal - Schiphol Airport,258
Rotterdam Centraal - Schiphol Airport...,248
Leeuwarden - Zwolle,112
Aachen Hbf - Heerlen,87
Winterswijk - Zutphen,43
Amersfoort - Ede-Wageningen,65
Breda - Dordrecht,68
Dordrecht - Roosendaal,103
's-Hertogenbosch - Tilburg,47
Breda - Rotterdam Centraal (HSL),165


Well, what a surprise that was!

Now, this was per line, what about per station? 
The data also provides the affected stations in each line using the `stationNames` column.

Let's do the same as before:

In [81]:
val byStation = df2
    .explode { stationNames }
    .groupBy { stationNames }

byStation.count().sortByDesc("count")

stationNames,count
Rotterdam Centraal,612
Schiphol Airport,493
Amsterdam Centraal,392
Utrecht Centraal,324
Amsterdam Sloterdijk,308
Breda,289
Arnhem Centraal,264
Zwolle,260
Leiden Centraal,248
Amersfoort Centraal,221


Interesting! We have another 'winner'.

Let's get some more information about the duration of the delay, because just a count doesn't tell the whole story.
We can `describe()` the `duration` column to get statistical details about it.

In [82]:
byStation.aggregate {
    duration.describe().first() into "duration"
}

stationNames,duration,,,,,,,,,
,name,type,count,unique,nulls,top,freq,min,median,max
Amsterdam Centraal,duration,Duration,392,215,0,37m,7,0s,1h 10m,5d 1h 54m
Amsterdam Lelylaan,duration,Duration,198,140,0,17m,5,1m,1h 6m,2d 4h 33m
Amsterdam Sloterdijk,duration,Duration,308,176,0,31m,8,0s,1h 3m,2d 4h 33m
Rotterdam Centraal,duration,Duration,612,226,0,6m,15,0s,42m,13d 7h 1m
Schiphol Airport,duration,Duration,493,220,0,6m,14,0s,48m,13d 7h 1m
Heerenveen,duration,Duration,33,32,0,24m,2,2m,51m,14h 44m
Wolvega,duration,Duration,31,29,0,21m,2,2m,51m,14h 44m
Heerenveen IJsstadion,duration,Duration,26,26,0,24m,1,2m,50m,14h 44m
Aachen Hbf,duration,Duration,33,31,0,9m,2,3m,10h 43m,7d 16h 26m


I don't know about you, but this requires some visualization, doesn't it?

Let's use Kandy, as it has excellent integration with notebooks and DataFrame.

In [83]:
%use kandy

Let's take a look at the examples: https://kotlin.github.io/kandy/examples.html

Looks like a boxplot can best show the results of a top-10 "worst" stations.

In [84]:
val top10 =
    byStation.sortByGroupDesc {
        count()
//    durationMinutes.mean()
//    count() * durationMinutes.median()
//    count() * durationMinutes.mean()
    }
        .filter { it.index() < 10 }
        .concat()

top10

id,lines,linesId,stationNames,stationCodes,statisticalCause,causeGroup,startTime,endTime,durationMinutes,duration,date
45999,[Amsterdam Centraal - Schiphol Airpor...,"[24, 32]",Rotterdam Centraal,"[ASD, ASDL, ASS, RTD, SHL]",points failure,infrastructure,2023-01-01T08:19:26,2023-01-01T22:43:08,864,14h 24m,2023-01-01
46007,[Breda - Rotterdam Centraal (HSL)],[15],Rotterdam Centraal,"[BD, RTD]",stranded train,rolling stock,2023-01-02T11:50:11,2023-01-02T12:25:39,35,35m,2023-01-02
46008,[Amsterdam Centraal - Schiphol Airpor...,"[24, 32]",Rotterdam Centraal,"[ASD, ASDL, ASS, RTD, SHL]",broken down train,rolling stock,2023-01-02T12:40:11,2023-01-02T13:08:08,28,28m,2023-01-02
46022,[Rotterdam Centraal - Schiphol Airpor...,[24],Rotterdam Centraal,"[RTD, SHL]",broken down train,rolling stock,2023-01-03T14:54:42,2023-01-03T15:02:54,8,8m,2023-01-03
46030,[Breda - Rotterdam Centraal (HSL)],[15],Rotterdam Centraal,"[BD, RTD]",person on the railway track,external,2023-01-04T09:15:07,2023-01-04T12:37:35,202,3h 22m,2023-01-04
46031,[Dordrecht - Rotterdam Centraal],[16],Rotterdam Centraal,"[BRD, DDR, RLB, RTB, RTD, RTST, RTZ, ...",person on the railway track,external,2023-01-04T09:15:16,2023-01-04T12:37:52,203,3h 23m,2023-01-04
46040,[Rotterdam Centraal - Schiphol Airpor...,[24],Rotterdam Centraal,"[RTD, SHL]",disruption elsewhere,logistical,2023-01-05T16:42:56,2023-01-05T17:17:38,35,35m,2023-01-05
46071,[Amsterdam Centraal - Schiphol Airpor...,"[24, 32]",Rotterdam Centraal,"[ASD, ASDL, ASS, RTD, SHL]",broken down train,rolling stock,2023-01-09T18:37:38,2023-01-09T19:08:17,31,31m,2023-01-09
46074,[Breda - Rotterdam Centraal (HSL)],[15],Rotterdam Centraal,"[BD, RTD]",broken down train,rolling stock,2023-01-09T22:02:46,2023-01-09T22:02:59,0,0s,2023-01-09
46076,[Dordrecht - Rotterdam Centraal],[16],Rotterdam Centraal,"[BRD, DDR, RLB, RTB, RTD, RTST, RTZ, ...",over-running engineering works,engineering work,2023-01-10T05:34:30,2023-01-10T12:00:32,386,6h 26m,2023-01-10


In [85]:
top10.plot {
    boxplot(x = stationNames, y = durationMinutes) {
        boxes.fillColor(stationNames.distinct()) {
            legend.type = LegendType.None
        }
        y {
            scale = continuous(transform = Transformation.LOG10)
        }
    }

    layout.size = 1000 to 500
}

<head>
 <meta charset="UTF-8">
 <style> html, body { margin: 0; overflow: hidden; } </style>
 <script type="text/javascript" data-lets-plot-script="library" src="https://cdn.jsdelivr.net/gh/JetBrains/lets-plot@v4.2.0/js-package/distr/lets-plot.min.js"></script>
 </head>
 <body>
 <div id="ha636x"></div>
 <script type="text/javascript" data-lets-plot-script="plot">
 var plotSpec={
"mapping":{
},
"data":{
},
"ggsize":{
"width":1000.0,
"height":500.0
},
"kind":"plot",
"scales":[{
"aesthetic":"x",
"name":"stationNames",
"limits":[null,null]
},{
"aesthetic":"y",
"name":"durationMinutes",
"limits":[null,null]
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"fill",
"discrete":true,
"guide":"none"
},{
"aesthetic":"x",
"discrete":true
},{
"aesthetic":"y",
"limits":[null,null]
},{
"aesthetic":"y",
"limits":[null,null],
"trans":"LOG10"
}],
"layers":[{
"mapping":{
"x":"x",
"ymin":"min",
"lower":"lower",
"middle":"middle",
"upper":"upper",
"ymax":"max",
"fill":"stationNames",
"y":"y"
},
"stat":"identity",
"data":{
"min":[0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,1.0],
"middle":[42.0,48.0,70.0,72.5,63.0,37.0,72.5,112.0,52.5,70.0],
"max":[228.0,293.0,395.0,355.0,331.0,210.0,505.0,688.0,375.0,346.0],
"stationNames":["Rotterdam Centraal","Schiphol Airport","Amsterdam Centraal","Utrecht Centraal","Amsterdam Sloterdijk","Breda","Arnhem Centraal","Zwolle","Leiden Centraal","Amersfoort Centraal"],
"lower":[19.0,18.0,31.0,27.0,29.25,13.0,26.0,33.0,23.25,24.5],
"upper":[104.0,128.5,177.75,167.75,154.5,93.0,221.0,303.5,170.0,153.5],
"x":["Rotterdam Centraal","Schiphol Airport","Amsterdam Centraal","Utrecht Centraal","Amsterdam Sloterdijk","Breda","Arnhem Centraal","Zwolle","Leiden Centraal","Amersfoort Centraal"]
},
"sampling":"none",
"position":"identity",
"geom":"boxplot"
},{
"mapping":{
"x":"x",
"y":"y"
},
"stat":"identity",
"data":{
"x":["Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Rotterdam Centraal","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport","Schiphol Airport

## Do I get to complain about Dutch trains in a demo?